In [1]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score
from sklearn.model_selection import GridSearchCV

import numpy as np
import pandas as pd
import datetime as dt
from sqlalchemy import create_engine
import os 

# CHARGEMENT DES DONNEES 

In [2]:
dialect = 'mysql+pymysql'
user = 'root'
password = os.getenv('mdp_mySQL')
host = 'localhost'
port = 3306
database_idf = 'SNCF_TGV_TER'

engine_url = f"{dialect}://{user}:{password}@{host}:{port}/{database_idf}"
engine = create_engine(engine_url)

In [3]:
df_disruption = pd.read_sql('disruptions_tgv_ter',engine)
df_disruption

,id_disruption,vehicle_id,train_type,id_stop,name_stop,lon,lat,disruption_start,disruption_end,arrival_delay,departure_delay,cause_delay,data_date
0,300d66cc-2614-4087-89d7-89abe997e2cf,SNCF:2024-03-18:12244:1187:LongDistanceTrain,TGV,stop_point:SNCF:87391003:LongDistanceTrain,Paris - Montparnasse - Hall 1 & 2,2.320510,48.8412,2024-03-18 18:46:00,2024-03-18 21:49:00,50.0,50.0,Défaut d'alimentation électrique,2024-03-18
1,300d66cc-2614-4087-89d7-89abe997e2cf,SNCF:2024-03-18:12244:1187:LongDistanceTrain,TGV,stop_point:SNCF:87393702:LongDistanceTrain,Massy TGV,2.261250,48.7258,2024-03-18 18:46:00,2024-03-18 21:49:00,50.0,50.0,Défaut d'alimentation électrique,2024-03-18
2,300d66cc-2614-4087-89d7-89abe997e2cf,SNCF:2024-03-18:12244:1187:LongDistanceTrain,TGV,stop_point:SNCF:87581009:LongDistanceTrain,Bordeaux Saint-Jean,-0.556697,44.8259,2024-03-18 18:46:00,2024-03-18 21:49:00,0.0,0.0,Défaut d'alimentation électrique,2024-03-18
3,dc1eeb61-3ef0-4bfe-b63b-8d52ec7577c5,SNCF:2024-03-18:12254:1187:LongDistanceTrain,TGV,stop_point:SNCF:87391003:LongDistanceTrain,Paris - Montparnasse - Hall 1 & 2,2.320510,48.8412,2024-03-18 10:46:00,2024-03-18 12:58:00,5.0,5.0,Régulation du trafic,2024-03-18
4,dc1eeb61-3ef0-4bfe-b63b-8d52ec7577c5,SNCF:2024-03-18:12254:1187:LongDistanceTrain,TGV,stop_point:SNCF:87581009:LongDistanceTrain,Bordeaux Saint-Jean,-0.556697,44.8259,2024-03-18 10:46:00,2024-03-18 12:58:00,0.0,0.0,Régulation du trafic,2024-03-18
...,...,...,...,...,...,...,...,...,...,...,...,...,...
35423,d79dcb9b-8f5f-4cfc-9a9f-f777544ea5e4,SNCF:2024-03-23:9890:1187:LongDistanceTrain,TGV,stop_point:SNCF:87171926:LongDistanceTrain,Champagne-Ardenne TGV,3.994520,49.2148,2024-03-23 05:59:00,2024-03-23 10:08:00,0.0,0.0,Réutilisation d'un train,2024-03-23
35424,d79dcb9b-8f5f-4cfc-9a9f-f777544ea5e4,SNCF:2024-03-23:9890:1187:LongDistanceTrain,TGV,stop_point:SNCF:87212027:LongDistanceTrain,Strasbourg,7.734790,48.5845,2024-03-23 05:59:00,2024-03-23 10:08:00,0.0,0.0,Réutilisation d'un train,2024-03-23
35425,d79dcb9b-8f5f-4cfc-9a9f-f777544ea5e4,SNCF:2024-03-23:9890:1187:LongDistanceTrain,TGV,stop_point:SNCF:87223263:LongDistanceTrain,Lille Europe,3.075720,50.6394,2024-03-23 05:59:00,2024-03-23 10:08:00,25.0,NaN,Réutilisation d'un train,2024-03-23
35426,d79dcb9b-8f5f-4cfc-9a9f-f777544ea5e4,SNCF:2024-03-23:9890:1187:LongDistanceTrain,TGV,stop_point:SNCF:87271494:LongDistanceTrain,Aéroport Charles de Gaulle 2 TGV,2.570810,49.0036,2024-03-23 05:59:00,2024-03-23 10:08:00,0.0,10.0,Réutilisation d'un train,2024-03-23


In [4]:
df_vehicle = pd.read_sql('vehicle_journeys_tgv_ter', engine)
df_vehicle['vehicle_id'] = df_vehicle['vehicle_id'].apply(lambda x: ':'.join(x.split(':')[1:]))
df_vehicle

,vehicle_id,route_id,time_begin,time_end,train_type,id_disruption,data_date
0,SNCF:2024-03-18:117752:1187:LongDistanceTrain,route:SNCF:FR:Line::B10C45A0-C32C-4232-85F2-4B...,05:00:00,07:19:00,Train grande vitesse,0f4e5f2c-a148-438f-b7cb-71b80199e8d4,2024-03-18
1,SNCF:2024-03-18:12244:1187:LongDistanceTrain,route:SNCF:CSR:421400,18:46:00,20:59:00,Train grande vitesse,300d66cc-2614-4087-89d7-89abe997e2cf,2024-03-18
2,SNCF:2024-03-18:12251:1187:LongDistanceTrain,route:SNCF:CSR:421400,07:04:00,09:14:00,Train grande vitesse,None,2024-03-18
3,SNCF:2024-03-18:12254:1187:LongDistanceTrain,route:SNCF:CSR:421400,10:46:00,12:53:00,Train grande vitesse,dc1eeb61-3ef0-4bfe-b63b-8d52ec7577c5,2024-03-18
4,SNCF:2024-03-18:12255:1187:LongDistanceTrain,route:SNCF:CSR:421400,11:04:00,13:14:00,Train grande vitesse,None,2024-03-18
...,...,...,...,...,...,...,...
50261,SNCF:2024-03-23:TC2324:1187:Train,route:SNCF:1187-87755009-87755371,11:00:00,11:22:00,TER / Intercités,None,2024-03-23
50262,SNCF:2024-03-23:TC2325:1187:Train,route:SNCF:1187-87755009-87755371,13:50:00,14:13:00,TER / Intercités,None,2024-03-23
50263,SNCF:2024-03-23:TC2326:1187:Train,route:SNCF:1187-87755009-87755371,15:20:00,15:42:00,TER / Intercités,None,2024-03-23
50264,SNCF:2024-03-23:TC2327:1187:Train,route:SNCF:1187-87755009-87755371,17:20:00,17:46:00,TER / Intercités,None,2024-03-23


In [5]:
df_lines = pd.read_sql_table('pt_lines_tgv_ter', engine)
df_lines

,route_id,route_name,train_type,network_name,opening_time,closing_time
0,route:SNCF:0018-80100537-87271007,Dortmund Hbf - Paris Nord,Train grande vitesse,Eurostar,05:18:00,12:21:00
1,route:SNCF:0018-80101840-87271007,Essen Hbf - Paris Nord,Train grande vitesse,Eurostar,07:35:00,22:14:00
2,route:SNCF:0018-80154583-87271007,Köln - Hauptbahnhof - Paris Nord,Train grande vitesse,Eurostar,05:21:00,23:14:00
3,route:SNCF:0018-84000588-87111849,Amsterdam Centraal - Marne-la-Vallée - Chessy,Train grande vitesse,Eurostar,06:47:00,11:05:00
4,route:SNCF:0018-84000588-87271007,Amsterdam Centraal - Paris Nord,Train grande vitesse,Eurostar,06:11:00,23:43:00
...,...,...,...,...,...,...
363,route:stop_area:SNCF:87113001_stop_area:SNCF:8...,Paris Est - Strasbourg,Train grande vitesse,additional service,None,None
364,route:stop_area:SNCF:87394007_stop_area:SNCF:8...,Chartres - Versailles Chantiers,Train grande vitesse,additional service,None,None
365,route:stop_area:SNCF:87411017_stop_area:SNCF:8...,Rouen Rive Droite - Paris Saint-Lazare,Train grande vitesse,additional service,None,None
366,route:stop_area:SNCF:87411017_stop_area:SNCF:8...,Rouen Rive Droite - Le Havre,Train grande vitesse,additional service,None,None


# PREPARATION DES DONNEES 

## Merge des tables 

Probleme que je ne comprend pas encore : il y a beaucoup plus de perturbation dans df_vehicle que dans df_disruption

In [6]:
df_merge_inter = pd.merge(df_disruption.drop(['data_date','train_type'],axis=1), df_vehicle, on=['vehicle_id','id_disruption'], how='left')
df_merge = pd.merge(df_merge_inter, df_lines.drop('train_type', axis=1), on='route_id', how='left')

In [7]:
df_merge_clean = df_merge.dropna(subset=['arrival_delay', 'departure_delay'])
df_merge_clean

,id_disruption,vehicle_id,id_stop,name_stop,lon,lat,disruption_start,disruption_end,arrival_delay,departure_delay,cause_delay,route_id,time_begin,time_end,train_type,data_date,route_name,network_name,opening_time,closing_time
0,300d66cc-2614-4087-89d7-89abe997e2cf,SNCF:2024-03-18:12244:1187:LongDistanceTrain,stop_point:SNCF:87391003:LongDistanceTrain,Paris - Montparnasse - Hall 1 & 2,2.320510,48.8412,2024-03-18 18:46:00,2024-03-18 21:49:00,50.0,50.0,Défaut d'alimentation électrique,route:SNCF:CSR:421400,18:46:00,20:59:00,Train grande vitesse,2024-03-18,Paris - Montparnasse - Hall 1 & 2 - Bordeaux S...,TGV INOUI,05:17:00,00:46:00
1,300d66cc-2614-4087-89d7-89abe997e2cf,SNCF:2024-03-18:12244:1187:LongDistanceTrain,stop_point:SNCF:87393702:LongDistanceTrain,Massy TGV,2.261250,48.7258,2024-03-18 18:46:00,2024-03-18 21:49:00,50.0,50.0,Défaut d'alimentation électrique,route:SNCF:CSR:421400,18:46:00,20:59:00,Train grande vitesse,2024-03-18,Paris - Montparnasse - Hall 1 & 2 - Bordeaux S...,TGV INOUI,05:17:00,00:46:00
2,300d66cc-2614-4087-89d7-89abe997e2cf,SNCF:2024-03-18:12244:1187:LongDistanceTrain,stop_point:SNCF:87581009:LongDistanceTrain,Bordeaux Saint-Jean,-0.556697,44.8259,2024-03-18 18:46:00,2024-03-18 21:49:00,0.0,0.0,Défaut d'alimentation électrique,route:SNCF:CSR:421400,18:46:00,20:59:00,Train grande vitesse,2024-03-18,Paris - Montparnasse - Hall 1 & 2 - Bordeaux S...,TGV INOUI,05:17:00,00:46:00
3,dc1eeb61-3ef0-4bfe-b63b-8d52ec7577c5,SNCF:2024-03-18:12254:1187:LongDistanceTrain,stop_point:SNCF:87391003:LongDistanceTrain,Paris - Montparnasse - Hall 1 & 2,2.320510,48.8412,2024-03-18 10:46:00,2024-03-18 12:58:00,5.0,5.0,Régulation du trafic,route:SNCF:CSR:421400,10:46:00,12:53:00,Train grande vitesse,2024-03-18,Paris - Montparnasse - Hall 1 & 2 - Bordeaux S...,TGV INOUI,05:17:00,00:46:00
4,dc1eeb61-3ef0-4bfe-b63b-8d52ec7577c5,SNCF:2024-03-18:12254:1187:LongDistanceTrain,stop_point:SNCF:87581009:LongDistanceTrain,Bordeaux Saint-Jean,-0.556697,44.8259,2024-03-18 10:46:00,2024-03-18 12:58:00,0.0,0.0,Régulation du trafic,route:SNCF:CSR:421400,10:46:00,12:53:00,Train grande vitesse,2024-03-18,Paris - Montparnasse - Hall 1 & 2 - Bordeaux S...,TGV INOUI,05:17:00,00:46:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35421,d79dcb9b-8f5f-4cfc-9a9f-f777544ea5e4,SNCF:2024-03-23:9890:1187:LongDistanceTrain,stop_point:SNCF:87142109:LongDistanceTrain,Lorraine TGV,6.169780,48.9477,2024-03-23 05:59:00,2024-03-23 10:08:00,0.0,0.0,Réutilisation d'un train,route:SNCF:CSR:005400,05:59:00,09:43:00,Train grande vitesse,2024-03-23,Strasbourg - Bruxelles-Midi / Brussel-Zuid,TGV INOUI,05:59:00,21:11:00
35422,d79dcb9b-8f5f-4cfc-9a9f-f777544ea5e4,SNCF:2024-03-23:9890:1187:LongDistanceTrain,stop_point:SNCF:87147322:LongDistanceTrain,Meuse TGV,5.271020,48.9786,2024-03-23 05:59:00,2024-03-23 10:08:00,0.0,0.0,Réutilisation d'un train,route:SNCF:CSR:005400,05:59:00,09:43:00,Train grande vitesse,2024-03-23,Strasbourg - Bruxelles-Midi / Brussel-Zuid,TGV INOUI,05:59:00,21:11:00
35423,d79dcb9b-8f5f-4cfc-9a9f-f777544ea5e4,SNCF:2024-03-23:9890:1187:LongDistanceTrain,stop_point:SNCF:87171926:LongDistanceTrain,Champagne-Ardenne TGV,3.994520,49.2148,2024-03-23 05:59:00,2024-03-23 10:08:00,0.0,0.0,Réutilisation d'un train,route:SNCF:CSR:005400,05:59:00,09:43:00,Train grande vitesse,2024-03-23,Strasbourg - Bruxelles-Midi / Brussel-Zuid,TGV INOUI,05:59:00,21:11:00
35424,d79dcb9b-8f5f-4cfc-9a9f-f777544ea5e4,SNCF:2024-03-23:9890:1187:LongDistanceTrain,stop_point:SNCF:87212027:LongDistanceTrain,Strasbourg,7.734790,48.5845,2024-03-23 05:59:00,2024-03-23 10:08:00,0.0,0.0,Réutilisation d'un train,route:SNCF:CSR:005400,05:59:00,09:43:00,Train grande vitesse,2024-03-23,Strasbourg - Bruxelles-Midi / Brussel-Zuid,TGV INOUI,05:59:00,21:11:00


### Rajout des trajets non perturbé

In [8]:
df_vehicle_add = df_vehicle[df_vehicle['id_disruption'].isna()]
df_vehicle_add

,vehicle_id,route_id,time_begin,time_end,train_type,id_disruption,data_date
2,SNCF:2024-03-18:12251:1187:LongDistanceTrain,route:SNCF:CSR:421400,07:04:00,09:14:00,Train grande vitesse,None,2024-03-18
4,SNCF:2024-03-18:12255:1187:LongDistanceTrain,route:SNCF:CSR:421400,11:04:00,13:14:00,Train grande vitesse,None,2024-03-18
6,SNCF:2024-03-18:12259:1187:LongDistanceTrain,route:SNCF:CSR:421400,12:04:00,14:14:00,Train grande vitesse,None,2024-03-18
7,SNCF:2024-03-18:12260:1187:LongDistanceTrain,route:SNCF:CSR:421400,14:46:00,17:03:00,Train grande vitesse,None,2024-03-18
9,SNCF:2024-03-18:12265:1187:LongDistanceTrain,route:SNCF:CSR:421400,13:58:00,16:14:00,Train grande vitesse,None,2024-03-18
...,...,...,...,...,...,...,...
50261,SNCF:2024-03-23:TC2324:1187:Train,route:SNCF:1187-87755009-87755371,11:00:00,11:22:00,TER / Intercités,None,2024-03-23
50262,SNCF:2024-03-23:TC2325:1187:Train,route:SNCF:1187-87755009-87755371,13:50:00,14:13:00,TER / Intercités,None,2024-03-23
50263,SNCF:2024-03-23:TC2326:1187:Train,route:SNCF:1187-87755009-87755371,15:20:00,15:42:00,TER / Intercités,None,2024-03-23
50264,SNCF:2024-03-23:TC2327:1187:Train,route:SNCF:1187-87755009-87755371,17:20:00,17:46:00,TER / Intercités,None,2024-03-23


In [9]:
df_vehicle_add.info()

<class 'pandas.core.frame.DataFrame'>
Index: 43452 entries, 2 to 50265
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   vehicle_id     43452 non-null  object        
 1   route_id       43452 non-null  object        
 2   time_begin     43452 non-null  object        
 3   time_end       43452 non-null  object        
 4   train_type     43452 non-null  object        
 5   id_disruption  0 non-null      object        
 6   data_date      43452 non-null  datetime64[ns]
dtypes: datetime64[ns](1), object(6)
memory usage: 2.7+ MB


In [10]:
df_final = pd.concat([df_merge_inter, df_vehicle_add], ignore_index=True)
df_final

,id_disruption,vehicle_id,id_stop,name_stop,lon,lat,disruption_start,disruption_end,arrival_delay,departure_delay,cause_delay,route_id,time_begin,time_end,train_type,data_date
0,300d66cc-2614-4087-89d7-89abe997e2cf,SNCF:2024-03-18:12244:1187:LongDistanceTrain,stop_point:SNCF:87391003:LongDistanceTrain,Paris - Montparnasse - Hall 1 & 2,2.320510,48.8412,2024-03-18 18:46:00,2024-03-18 21:49:00,50.0,50.0,Défaut d'alimentation électrique,route:SNCF:CSR:421400,18:46:00,20:59:00,Train grande vitesse,2024-03-18
1,300d66cc-2614-4087-89d7-89abe997e2cf,SNCF:2024-03-18:12244:1187:LongDistanceTrain,stop_point:SNCF:87393702:LongDistanceTrain,Massy TGV,2.261250,48.7258,2024-03-18 18:46:00,2024-03-18 21:49:00,50.0,50.0,Défaut d'alimentation électrique,route:SNCF:CSR:421400,18:46:00,20:59:00,Train grande vitesse,2024-03-18
2,300d66cc-2614-4087-89d7-89abe997e2cf,SNCF:2024-03-18:12244:1187:LongDistanceTrain,stop_point:SNCF:87581009:LongDistanceTrain,Bordeaux Saint-Jean,-0.556697,44.8259,2024-03-18 18:46:00,2024-03-18 21:49:00,0.0,0.0,Défaut d'alimentation électrique,route:SNCF:CSR:421400,18:46:00,20:59:00,Train grande vitesse,2024-03-18
3,dc1eeb61-3ef0-4bfe-b63b-8d52ec7577c5,SNCF:2024-03-18:12254:1187:LongDistanceTrain,stop_point:SNCF:87391003:LongDistanceTrain,Paris - Montparnasse - Hall 1 & 2,2.320510,48.8412,2024-03-18 10:46:00,2024-03-18 12:58:00,5.0,5.0,Régulation du trafic,route:SNCF:CSR:421400,10:46:00,12:53:00,Train grande vitesse,2024-03-18
4,dc1eeb61-3ef0-4bfe-b63b-8d52ec7577c5,SNCF:2024-03-18:12254:1187:LongDistanceTrain,stop_point:SNCF:87581009:LongDistanceTrain,Bordeaux Saint-Jean,-0.556697,44.8259,2024-03-18 10:46:00,2024-03-18 12:58:00,0.0,0.0,Régulation du trafic,route:SNCF:CSR:421400,10:46:00,12:53:00,Train grande vitesse,2024-03-18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78875,None,SNCF:2024-03-23:TC2324:1187:Train,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,route:SNCF:1187-87755009-87755371,11:00:00,11:22:00,TER / Intercités,2024-03-23
78876,None,SNCF:2024-03-23:TC2325:1187:Train,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,route:SNCF:1187-87755009-87755371,13:50:00,14:13:00,TER / Intercités,2024-03-23
78877,None,SNCF:2024-03-23:TC2326:1187:Train,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,route:SNCF:1187-87755009-87755371,15:20:00,15:42:00,TER / Intercités,2024-03-23
78878,None,SNCF:2024-03-23:TC2327:1187:Train,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,route:SNCF:1187-87755009-87755371,17:20:00,17:46:00,TER / Intercités,2024-03-23


## Choix des variables

In [11]:
df_final['disrupted'] = np.where(df_final['id_disruption'].isna(), 0, 1)

In [12]:
df_final['week_day'] = df_final['data_date'].dt.weekday

In [13]:
df_final = df_final[[#'id_disruption', 
        #'vehicle_id', 
        'id_stop', 
        #'name_stop', 
        #'lon', 
        #'lat',
        'disruption_start',
        'disruption_end', 
        'arrival_delay',
        'departure_delay', 
        'cause_delay',
        'route_id',
        'time_begin',
        'time_end',
        'train_type',
        #'data_date',
        'disrupted',
        'week_day']]

df_final

,id_stop,disruption_start,disruption_end,arrival_delay,departure_delay,cause_delay,route_id,time_begin,time_end,train_type,disrupted,week_day
0,stop_point:SNCF:87391003:LongDistanceTrain,2024-03-18 18:46:00,2024-03-18 21:49:00,50.0,50.0,Défaut d'alimentation électrique,route:SNCF:CSR:421400,18:46:00,20:59:00,Train grande vitesse,1,0
1,stop_point:SNCF:87393702:LongDistanceTrain,2024-03-18 18:46:00,2024-03-18 21:49:00,50.0,50.0,Défaut d'alimentation électrique,route:SNCF:CSR:421400,18:46:00,20:59:00,Train grande vitesse,1,0
2,stop_point:SNCF:87581009:LongDistanceTrain,2024-03-18 18:46:00,2024-03-18 21:49:00,0.0,0.0,Défaut d'alimentation électrique,route:SNCF:CSR:421400,18:46:00,20:59:00,Train grande vitesse,1,0
3,stop_point:SNCF:87391003:LongDistanceTrain,2024-03-18 10:46:00,2024-03-18 12:58:00,5.0,5.0,Régulation du trafic,route:SNCF:CSR:421400,10:46:00,12:53:00,Train grande vitesse,1,0
4,stop_point:SNCF:87581009:LongDistanceTrain,2024-03-18 10:46:00,2024-03-18 12:58:00,0.0,0.0,Régulation du trafic,route:SNCF:CSR:421400,10:46:00,12:53:00,Train grande vitesse,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...
78875,NaN,NaT,NaT,NaN,NaN,NaN,route:SNCF:1187-87755009-87755371,11:00:00,11:22:00,TER / Intercités,0,5
78876,NaN,NaT,NaT,NaN,NaN,NaN,route:SNCF:1187-87755009-87755371,13:50:00,14:13:00,TER / Intercités,0,5
78877,NaN,NaT,NaT,NaN,NaN,NaN,route:SNCF:1187-87755009-87755371,15:20:00,15:42:00,TER / Intercités,0,5
78878,NaN,NaT,NaT,NaN,NaN,NaN,route:SNCF:1187-87755009-87755371,17:20:00,17:46:00,TER / Intercités,0,5


# RANDOM FOREST (CLASSIFICATION)